# Data Preprocessing and Transformation Assignment


In this assignment, you will practice data preprocessing and transformation techniques usingdataset from a telecom company. The dataset is stored in an SQLite database and contains two tables: customer demographics and subscription details.

Your tasks are to:

1. Read the `.db` database file.
2. Merge the two tables into one DataFrame.
3. Explore the merged DataFrame.
4. Assess and handle missing values.
5. Assess and filter out outliers.
6. Impute missing data.
7. Apply ordinal encoding to ordinal variables.
8. Apply one-hot encoding to nominal variables.
9. Merge the encoded features into the final DataFrame.
10. **For every task** create a markdown cell  and explain what you have done and also the results

> Make sure the notebook is clear of syntax error, do not output unnecessary data, keep clean and neat.




## Task 1: Read the Database (`telecom_data.db`)

In [151]:
# import pandas and splite3 database into google collabs and establishing a connection
import sqlite3
import pandas as pd
conn = sqlite3.connect('/content/telecom_data.db')

In [152]:
# To determine what tables are in the sqlite3 db, this will help with pulling the tables into pandas
query = "SELECT name FROM sqlite_master WHERE type = 'table';"
tables_df = pd.read_sql_query(query, conn)
print(tables_df)

           name
0      customer
1  subscription


In [153]:
# Pulling the customer table from the sqlite3 to be used under pandas
customer_demographic_df = pd.read_sql_query('SELECT * FROM customer', conn)

In [154]:
# viewing the top 5 instances to understand the columns
customer_demographic_df.head()

,customer_id,age,gender,income,region
0,1,56.0,Male,44900.0,North
1,2,69.0,Male,38000.0,North
2,3,46.0,Male,47600.0,East
3,4,32.0,Other,56100.0,South
4,5,60.0,Male,78300.0,West


In [155]:
# Pulling the subscription table from sqlite3 to be used under pandas
subscriptions_df = pd.read_sql_query('SELECT * FROM subscription', conn)

In [156]:
# viewing the top 5 instances to understand the columns
subscriptions_df.head()

,customer_id,plan,monthly_charges,contract
0,1,Standard,99.97,Two year
1,2,Basic,99.70,Two year
2,3,Premium,59.99,One year
3,4,Basic,79.21,Two year
4,5,Basic,95.03,Two year


## Task 2: Merge the Two Tables into One DataFrame called `telecom_df`

In [157]:
# merging both df into one using the customer_id as the related column, using an inner merge
telecom_df = pd.merge(customer_demographic_df, subscriptions_df, on='customer_id', how='inner')

In [158]:
print(telecom_df)

    customer_id   age  gender   income region      plan  monthly_charges  \
0             1  56.0    Male  44900.0  North  Standard            99.97   
1             2  69.0    Male  38000.0  North     Basic            99.70   
2             3  46.0    Male  47600.0   East   Premium            59.99   
3             4  32.0   Other  56100.0  South     Basic            79.21   
4             5  60.0    Male  78300.0   West     Basic            95.03   
..          ...   ...     ...      ...    ...       ...              ...   
95           96  42.0   Other  52300.0  South     Basic            51.63   
96           97  62.0    Male      NaN   East   Premium            77.27   
97           98  58.0  Female  32900.0   East  Standard            13.30   
98           99  46.0    Male  55400.0  North     Basic            32.72   
99          100  32.0    Male  58400.0   West     Basic            74.20   

          contract  
0         Two year  
1         Two year  
2         One year  
3  

## Task 3: Explore the Merged DataFrame
- use info, describe,..

In [159]:
# .info() will inform me the different types of data that are assigned to each column also tell me if there is null entries
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      100 non-null    int64  
 1   age              90 non-null     float64
 2   gender           100 non-null    object 
 3   income           95 non-null     float64
 4   region           100 non-null    object 
 5   plan             100 non-null    object 
 6   monthly_charges  100 non-null    float64
 7   contract         100 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 6.4+ KB


In [160]:
# This helps me narrow down where my null is in this dataset
telecom_df.isnull().sum()

,0
customer_id,0
age,10
gender,0
income,5
region,0
plan,0
monthly_charges,0
contract,0


In [161]:
# being Numerical data, using the median for age seems fit if I need to impute missing data
telecom_df['age'].median()

41.5

In [162]:
# being Categorical Nominal data, using unique to find the different values will help me when start encoding
telecom_df['gender'].unique()

array(['Male', 'Other', 'Female'], dtype=object)

In [163]:
# being Numerical data, using the median for income seems fit if I need to impute missing data
telecom_df['income'].median()

52300.0

In [164]:
# being Categorical Nominal data, using unique to find the different values will help me when start encoding
telecom_df['region'].unique()

array(['North', 'East', 'South', 'West'], dtype=object)

In [165]:
# being Categorical Ordinal data, using unique to find the different values will help me when start encoding
telecom_df['plan'].unique()

array(['Standard', 'Basic', 'Premium'], dtype=object)

In [166]:
# being Numerical data, using the median for monthly_charges seems fit if I need to impute missing data
telecom_df['monthly_charges'].median()

52.144999999999996

In [167]:
# being Categorical Ordinal data, using unique to find the different values will help me when start encoding
telecom_df['contract'].unique()

array(['Two year', 'One year', 'Month-to-month'], dtype=object)

In [168]:
# .describe() will inform me of the different int values of my data set
telecom_df.describe()

,customer_id,age,income,monthly_charges
count,100.000000,90.000000,95.000000,100.000000
mean,50.500000,43.277778,50407.368421,68.255700
std,29.011492,14.979407,14301.742507,113.586475
min,1.000000,19.000000,21200.000000,11.310000
25%,25.750000,31.250000,37900.000000,28.772500
50%,50.500000,41.500000,52300.000000,52.145000
75%,75.250000,56.000000,58100.000000,74.967500
max,100.000000,69.000000,90800.000000,954.600000


In [169]:
# shape will tell me the number of instances and columns are present in my data set
telecom_df.shape

(100, 8)

## Task 4: Assess Missing Values

In [170]:
# .info() will inform me the different types of data that are assigned to each column also tell me if there is null entries
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      100 non-null    int64  
 1   age              90 non-null     float64
 2   gender           100 non-null    object 
 3   income           95 non-null     float64
 4   region           100 non-null    object 
 5   plan             100 non-null    object 
 6   monthly_charges  100 non-null    float64
 7   contract         100 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 6.4+ KB


In [171]:
# This helps me narrow down where my null is in this dataset
telecom_df.isnull().sum()

,0
customer_id,0
age,10
gender,0
income,5
region,0
plan,0
monthly_charges,0
contract,0


## Task 5: Impute Missing Data

In [172]:
# due to the missing entries for age, this code will enter in the median of all the ages in the dataset in place of a null entry for age
telecom_df['age'].fillna(telecom_df['age'].median(), inplace=True)

In [173]:
# due to the missing entries for income, this code will enter in the median of all the incomes in the dataset in place of a null entry for income
telecom_df['income'].fillna(telecom_df['income'].median(), inplace=True)

In [174]:
# using .isnull().sum() again to verify that the null entries are now filled with the median values of the perspective columns
telecom_df.isnull().sum()

,0
customer_id,0
age,0
gender,0
income,0
region,0
plan,0
monthly_charges,0
contract,0


## Task 6: Apply Ordinal Encoding to Ordinal Variables

In [175]:
# importing OrdinalEncoder to apply ordinal encoding to ordinal variables
from sklearn.preprocessing import OrdinalEncoder

In [176]:
# to determine the different values in this categorical ordinal dataset
telecom_df['plan'].unique()

array(['Standard', 'Basic', 'Premium'], dtype=object)

In [177]:
# Assign plan_encoder as the new way to determine gender by giving an int system
plan_encoder = OrdinalEncoder(categories=[['Standard', 'Basic', 'Premium']])

In [178]:
# inputting the new plan_encoder column into telecom_df
telecom_df['plan_encoder'] = plan_encoder.fit_transform(telecom_df['plan'].values.reshape(-1,1))

In [179]:
# provide a sample to verify plan_encoder is correct with original dataset
telecom_df.sample(5)

,customer_id,age,gender,income,region,plan,monthly_charges,contract,plan_encoder
81,82,41.5,Male,57200.0,South,Standard,66.51,Two year,0.0
39,40,69.0,Female,69600.0,East,Basic,56.26,One year,1.0
61,62,64.0,Other,37700.0,West,Premium,26.37,One year,2.0
0,1,56.0,Male,44900.0,North,Standard,99.97,Two year,0.0
93,94,41.0,Other,55800.0,South,Premium,65.64,One year,2.0


In [180]:
# to determine the different values in this categorical ordinal dataset
telecom_df['contract'].unique()

array(['Two year', 'One year', 'Month-to-month'], dtype=object)

In [181]:
contract_encoder = OrdinalEncoder(categories=[['Two year', 'One year', 'Month-to-month']])

In [182]:
telecom_df['contract_encoder'] = contract_encoder.fit_transform(telecom_df['contract'].values.reshape(-1,1))

In [183]:
telecom_df.sample(5)

,customer_id,age,gender,income,region,plan,monthly_charges,contract,plan_encoder,contract_encoder
89,90,33.0,Male,62800.0,South,Basic,26.61,Month-to-month,1.0,2.0
51,52,67.0,Other,53500.0,South,Basic,54.34,Month-to-month,1.0,2.0
58,59,24.0,Other,35400.0,North,Basic,23.67,One year,1.0,1.0
18,19,19.0,Male,61900.0,North,Basic,79.51,Month-to-month,1.0,2.0
92,93,64.0,Other,52600.0,North,Premium,53.61,One year,2.0,1.0


## Task 7: Apply One-Hot Encoding to Nominal Variables

In [184]:
# The categorical nominal variables are 'gender' and 'region'
# Applying one-hot encoding to'gender' and 'region'
ohe_gender = pd.get_dummies(telecom_df['gender'], prefix='gender').astype(int)
ohe_region = pd.get_dummies(telecom_df['region'], prefix='region').astype(int)

In [185]:
# this provides the ohe applied to the gender data
ohe_gender.head()

,gender_Female,gender_Male,gender_Other
0,0,1,0
1,0,1,0
2,0,1,0
3,0,0,1
4,0,1,0


In [186]:
# this provides the ohe applied to the region data
ohe_region.head()

,region_East,region_North,region_South,region_West
0,0,1,0,0
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,0,0,0,1


## Task 8: Merge Encoded Features into the Final DataFrame called `final_telcom_df`

In [187]:
# creating final_telecom_df by combining the new ohe_gender and ohe_region with the telecom_df
final_telecom_df = pd.concat([telecom_df, ohe_gender, ohe_region], axis=1)

In [188]:
# checking to see what columns i will need to drop
final_telecom_df.head()

,customer_id,age,gender,income,region,plan,monthly_charges,contract,plan_encoder,contract_encoder,gender_Female,gender_Male,gender_Other,region_East,region_North,region_South,region_West
0,1,56.0,Male,44900.0,North,Standard,99.97,Two year,0.0,0.0,0,1,0,0,1,0,0
1,2,69.0,Male,38000.0,North,Basic,99.70,Two year,1.0,0.0,0,1,0,0,1,0,0
2,3,46.0,Male,47600.0,East,Premium,59.99,One year,2.0,1.0,0,1,0,1,0,0,0
3,4,32.0,Other,56100.0,South,Basic,79.21,Two year,1.0,0.0,0,0,1,0,0,1,0
4,5,60.0,Male,78300.0,West,Basic,95.03,Two year,1.0,0.0,0,1,0,0,0,0,1


In [189]:
# dropping the columns 'gender', 'region', 'plan', 'contract'
final_telecom_df = final_telecom_df.drop(['gender', 'region','plan','contract'], axis=1)

In [190]:
# the final_telecom_df to only represented in int to have better analysis
final_telecom_df.head()

,customer_id,age,income,monthly_charges,plan_encoder,contract_encoder,gender_Female,gender_Male,gender_Other,region_East,region_North,region_South,region_West
0,1,56.0,44900.0,99.97,0.0,0.0,0,1,0,0,1,0,0
1,2,69.0,38000.0,99.70,1.0,0.0,0,1,0,0,1,0,0
2,3,46.0,47600.0,59.99,2.0,1.0,0,1,0,1,0,0,0
3,4,32.0,56100.0,79.21,1.0,0.0,0,0,1,0,0,1,0
4,5,60.0,78300.0,95.03,1.0,0.0,0,1,0,0,0,0,1
